In [ ]:
pip install numpy pandas matplotlib scikit-learn scipy tensorflow


### GPU Activation 
*if GPU available*

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from itertools import product
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import csv
import matplotlib.dates as mdates



### Data Import

In [2]:
df_wind =pd.read_csv('https://drive.google.com/uc?id=1sbFAHLYmyOBhHZ8yHqct2yhViRRV9r3u')

In [3]:
df_wind['TimeStamp'] = pd.to_datetime(df_wind['TimeStamp'], format='mixed', errors ='coerce') #Change TimeStamp to datetime format
df_wind.info()
df_wind.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407140 entries, 0 to 407139
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   TimeStamp         407140 non-null  datetime64[ns]
 1   WindSpeed_Mean    407140 non-null  float64       
 2   VoltL1_Mean       407140 non-null  float64       
 3   VoltL2_Mean       407140 non-null  float64       
 4   VoltL3_Mean       407140 non-null  float64       
 5   ActivePower_Mean  407140 non-null  float64       
 6   CurrentL1_Mean    407140 non-null  float64       
 7   CurrentL2_Mean    407140 non-null  float64       
 8   CurrentL3_Mean    407140 non-null  float64       
 9   PowerFactor_Mean  407140 non-null  float64       
 10  Frequency_Mean    407140 non-null  float64       
 11  WindSpeed_Bin     407140 non-null  float64       
dtypes: datetime64[ns](1), float64(11)
memory usage: 37.3 MB


,TimeStamp,WindSpeed_Mean,VoltL1_Mean,VoltL2_Mean,VoltL3_Mean,ActivePower_Mean,CurrentL1_Mean,CurrentL2_Mean,CurrentL3_Mean,PowerFactor_Mean,Frequency_Mean,WindSpeed_Bin
0,2016-01-01 00:00:00,6.785655,392.817993,393.226410,392.404510,521.591797,588.076599,588.153687,588.288208,0.735540,50.030361,6.8
1,2016-01-01 00:10:00,6.085925,392.750610,393.095093,392.344208,352.522308,491.073486,491.226807,491.788910,0.564326,49.993130,6.1
2,2016-01-01 00:20:00,5.824687,393.093109,393.321503,392.433807,294.078888,472.158508,472.661804,472.771088,0.512421,49.952190,5.8
3,2016-01-01 00:30:00,7.100197,393.863586,394.046600,393.028595,587.302795,674.494019,674.627991,674.614319,0.705167,49.975941,7.1
4,2016-01-01 00:40:00,8.232280,394.487793,394.654785,393.713715,881.533325,866.352600,866.474121,865.911682,0.840677,49.930500,8.2


### Create Cyclical Time Features 
To help the LSTM model learn daily patterns in wind turbine power generation,  cyclical time features were created from the timestamp. Specifically, the hour and minute were transformed into sine and cosine values to represent the circular nature of time — for example, hour 23 and hour 0 are numerically far apart but temporally adjacent.

These sinusoidal features preserve the continuity of time and allow the model to capture periodic trends, such as changes in power output throughout the day. Without this transformation, the model might interpret 23:00 and 00:00 as completely unrelated, which could impair its ability to learn temporal dependencies.

In [5]:
def create_time_features(df, timestamp_col): 
    df['hour'] = df[timestamp_col].dt.hour
    df['minute'] = df[timestamp_col].dt.minute
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    minutes = df['hour'] * 60 + df['minute']
    df['minute_sin'] = np.sin(2 * np.pi * minutes / 1440)
    df['minute_cos'] = np.cos(2 * np.pi * minutes / 1440)

    return df.drop(columns=['hour', 'minute'])

### Pre-Processing

The below class handles preprocessing for the LSTM model by scaling input features, creating sliding time windows (sequences), and reshaping the data into the required 3D format. It also supports reversing the scaling for interpretation of predicted values.

In [6]:
class LSTMPreprocessor:
#Initialisation
    def __init__(self, feature_cols, target_col, sequence_length=144):
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.sequence_length = sequence_length
        self.scaler = MinMaxScaler() 

#Data Normalisation
    def fit(self, df):
        self.scaler.fit(df[self.feature_cols + [self.target_col]])

#Applies Normalisation Scaler
    def transform(self, df):
        scaled = self.scaler.transform(df[self.feature_cols + [self.target_col]])
        X, y = self.make_sequences(scaled)
        return X, y

#Creates the Sequence Length, the 'sliding window' for the LSTM 
    def make_sequences(self, data):
        X, y = [], []
        for i in range(len(data) - self.sequence_length + 1):
            # Inputs: rows i … i+sequence_length-1 (all features except final column is target)
            X.append(data[i : i+self.sequence_length, :-1])
            # Target: the power at the last row of that window (nowcast)
            y.append(data[i + self.sequence_length - 1, -1])
        return np.array(X), np.array(y)


#Apply on predictions - Scale Active Power back to orginal values
    def inverse_transform(self, y_scaled):
    # Ensure input is a 1D array
        y_scaled = np.array(y_scaled).reshape(-1)

        # Create dummy with the same number of columns used during scaling
        dummy = np.zeros((len(y_scaled), len(self.feature_cols) + 1))
        dummy[:, -1] = y_scaled

        # Inverse transform and extract only the target column
        return self.scaler.inverse_transform(dummy)[:, -1]


### LSTM Model

This custom LSTM model allows flexible configuration of multiple stacked LSTM layers with optional dropout for regularization. It outputs a single value predicting the next 10-minute power value. The architecture is modular, making it easy to adjust the number of layers and hidden units.

In [8]:
class CustomLSTM(tf.keras.Model):
    def __init__(self, hidden_size, num_layers, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs) # Pass any layer kwargs, like name of model to super
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.lstm_layers = []
        for i in range(num_layers):
            return_seq = (i < num_layers - 1)
            self.lstm_layers.append(   # Each LSTM layer is a keras Layer
                tf.keras.layers.LSTM(
                    hidden_size,
                    return_sequences=return_seq,
                    name=f"lstm_{i}"
                )
            )
            if dropout_rate > 0: #make dropout rate optional to add
                self.lstm_layers.append(
                    tf.keras.layers.Dropout(
                        dropout_rate,
                        name=f"dropout_{i}"
                    )
                )
        self.output_layer = tf.keras.layers.Dense(1, name="output_dense") # Dense layer is the output layer, 

    def call(self, x):
        for layer in self.lstm_layers:
            x = layer(x)
        return self.output_layer(x)


### LSTM Train

The below function trains the custom LSTM model using the Adam optimizer and Mean Squared Error (MSE) loss. It accepts training and validation sets, runs for a specified number of epochs and batch size, and returns the training history for later analysis. shuffle=False is used to preserve the time order of sequences, which is essential in time series modeling.

In [9]:
# Train Model with LSTM
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32, learning_rate=0.001):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse') #Use adam as inital optimser and learning rate initalisor, Mean Squared Error (mse) as loss function
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=epochs, batch_size=batch_size, verbose=1, shuffle = False) #Input epochs and batch size manually, use verbose to show progress bar during training
    return history


### Model Evaluation

This class handles model evaluation by calculating and reporting performance metrics across different dataset splits (e.g., train, validation, test). It first inversely transforms the scaled predictions using the fitted preprocessor, then computes standard regression metrics including RMSE, R², MAE, and SMAPE. Values close to zero, below a small threshold, are masked out to avoid instability in percentage-based metrics.

In [11]:
class EvaluationReport:
    def __init__(self, preprocessor, X_sets, target_col):

        self.preprocessor = preprocessor
        self.X_sets = X_sets
        self.target_col = target_col

    @staticmethod 
    def calculate_metrics(y_true, y_pred):
        eps = 1e-2
        mask = y_true > eps

        if np.sum(mask) == 0:
            print("[Warning] No valid values after masking low y_true values.")
            return {}

        y_true = y_true[mask]
        y_pred = y_pred[mask]
       
        #Evaluation Metrics 
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mean_y = np.mean(y_true)
        rmsepe_approx = (rmse / mean_y) * 100
        r2 = r2_score(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        smape = np.mean(2 * np.abs(y_pred - y_true)
                                 / (np.abs(y_true) + np.abs(y_pred) + eps)) * 100

        return {
            'RMSE': rmse,
            'RMSEPE_approx (%)': rmsepe_approx,
            'R2': r2,
            'MAE': mae ,
            'SMAPE (%)': smape
        }

    def evaluate(self, model):
        results = {}
        for name, (X, y, _) in self.X_sets.items():
            y_pred = model.predict(X, verbose=1).flatten()
            y_true_inv = self.preprocessor.inverse_transform(y)
            y_pred_inv = self.preprocessor.inverse_transform(y_pred)

            metrics = self.calculate_metrics(y_true_inv, y_pred_inv)

            print(f"\n=== {name} Metrics ===")
            print(len(y_true_inv), len(y_pred_inv))
            for m, v in metrics.items():
                print(f"{m.ljust(18)}: {v:8.2f}")
            

            results[name] = metrics

        return results



### Run Pipeline 

#### Pipeline Grid Search

This function runs a grid search over hyperparameters for LSTM-based nowcasting of wind turbine power. It automates the full pipeline: data splitting, time feature creation, scaling, sequence generation, model training with early stopping, and evaluation using multiple performance metrics. For each configuration, training, validation, and test metrics are logged to a CSV file, and loss curves are saved for comparison. This setup allows for robust model selection based on empirical performance across various parameter combinations.

In [13]:
def run_pipeline_with_grid_search(
    df, timestamp_col, feature_cols, target_col,
    sequence_length_options, hidden_size_options, learning_rate_options,
    num_layers_options, batch_size_options, dropout_rate_options, epoch_options,
    log_path='final_model_.csv', model_dir='final_model'
):
    def check_for_nans_and_infs(X, y, label):
        if np.isnan(X).any() or np.isnan(y).any():
            print(f"[ERROR] NaNs detected in {label}")
        if np.isinf(X).any() or np.isinf(y).any():
            print(f"[ERROR] Infs detected in {label}")

#Define function for TS splitting of dataset
    def time_series_split(df, timestamp_col, train_frac=0.7, val_frac=0.15, test_frac=0.15): 
        df = df.sort_values(timestamp_col).reset_index(drop=True)
        n = len(df)
        
        train_end = int(n * train_frac)
        val_end = train_end + int(n * val_frac)
        
        train_df = df.iloc[:train_end]
        val_df   = df.iloc[train_end:val_end]
        test_df  = df.iloc[val_end:]
        
        return train_df, val_df, test_df

#Plot loss curves of training and validation loss
    def plot_loss_curve(history, config_id): 
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f'Loss Curve - {config_id}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss (MSE)')
        plt.legend()
        plt.grid(True)
        os.makedirs('loss_plots', exist_ok=True)
        plt.savefig(f'loss_plots/loss_curve_{config_id}.png')
        plt.close()

    os.makedirs(model_dir, exist_ok=True)

    # Prepare log file
    dummy_metrics = EvaluationReport.calculate_metrics(np.array([10.0,12.0]), np.array([10.0, 11.5]))
    log_fields = [
        'timestamp', 'sequence_length', 'hidden_size', 'learning_rate',
        'dropout_rate', 'num_layers', 'batch_size', 'epochs'
    ]
    for split in ['Train', 'Validation', 'Test']:
        log_fields.extend([f"{split}_{k}" for k in dummy_metrics.keys()])


    if not os.path.exists(log_path):
        with open(log_path, 'w', newline='') as f:
            csv.writer(f).writerow(log_fields)

    df = df.copy()
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df = create_time_features(df, timestamp_col)

    for seq_len in sequence_length_options:
        print(f"Generating sequences for sequence_length = {seq_len}")

        # Split DataFrame based on timestamp
        train_df, val_df, test_df = time_series_split(df, 'TimeStamp')


        # Apply LSTMPreprocessor
        pre = LSTMPreprocessor(feature_cols, target_col, seq_len)
        pre.fit(train_df)

        #check for small sequences
        if any(len(split) <= seq_len for split in [train_df, val_df, test_df]):
            print(f"[SKIP] Skipping config {config_id} due to insufficient rows after split.")
            continue


        X_train, y_train = pre.transform(train_df)
        X_val, y_val     = pre.transform(val_df)
        X_test, y_test   = pre.transform(test_df)
        ts_train = train_df[timestamp_col].iloc[seq_len:].reset_index(drop=True)
        ts_val = val_df[timestamp_col].iloc[seq_len:].reset_index(drop=True)
        ts_test = test_df[timestamp_col].iloc[seq_len:].reset_index(drop=True)


        check_for_nans_and_infs(X_train, y_train, "Train Set")
        check_for_nans_and_infs(X_val, y_val, "Validation Set")

        for h_size, lr, nl, bs, dr, ep in product(
            hidden_size_options,
            learning_rate_options,
            num_layers_options,
            batch_size_options,
            dropout_rate_options,
            epoch_options
        ):
            config_id = f"seq{seq_len}_hid{h_size}_lr{lr}_nl{nl}_bs{bs}_dr{dr}_ep{ep}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            print(f"Trying config: {config_id}")

            model = CustomLSTM(hidden_size=h_size, num_layers=nl, dropout_rate=dr)
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')

            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=ep,
                batch_size=bs,
                callbacks=[
                    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                    ModelCheckpoint(os.path.join(model_dir, f"{config_id}.keras"), monitor='val_loss', save_best_only=True)
                ],
                verbose=1
            )

            plot_loss_curve(history, config_id)

            val_report = EvaluationReport(preprocessor=pre, X_sets={"Validation": (X_val, y_val, ts_val)}, target_col=target_col)
            train_report = EvaluationReport(preprocessor=pre, X_sets={"Train": (X_train, y_train, ts_train)}, target_col=target_col)
            test_report = EvaluationReport(preprocessor=pre, X_sets={"Test": (X_test, y_test, ts_test)}, target_col=target_col)
            val_metrics = val_report.evaluate(model)["Validation"]
            train_metrics = train_report.evaluate(model)["Train"]
            test_metrics = test_report.evaluate(model)["Test"]


            with open(log_path, 'a', newline='') as f:
                row = [
                    datetime.now(), seq_len, h_size, lr, dr, nl, bs, ep
                ]
                for metrics in [train_metrics, val_metrics, test_metrics]:
                    row.extend([metrics.get(k, None) for k in dummy_metrics.keys()])
                csv.writer(f).writerow(row)


    print("[INFO] Grid search completed. All configurations and validation metrics are logged.")
    print(f"[INFO] Check the log file here: {log_path}")

    return log_path


#### Random Grid Search of 30 Models

In [ ]:
import random
from itertools import product

#Define full hyper‐parameter pool
param_pools = {
    'sequence_length_options': [24, 96, 144, 150],
    'hidden_size_options':     [32, 64, 128, 256],
    'learning_rate_options':   [0.00001, 0.001, 0.0003],
    'num_layers_options':      [1, 2, 3],
    'batch_size_options':      [16, 32, 64],
    'dropout_rate_options':    [0.1, 0.2, 0.3],
    'epoch_options':           [10, 50, 150, 200, 300]
}

#Generate the full list of tuples, then samples 30 of them
all_configs = list(product(
    param_pools['sequence_length_options'],
    param_pools['hidden_size_options'],
    param_pools['learning_rate_options'],
    param_pools['num_layers_options'],
    param_pools['batch_size_options'],
    param_pools['dropout_rate_options'],
    param_pools['epoch_options'],
))
random.seed(42)
sampled = random.sample(all_configs, 30)

#Loop over each sampled tuple, calling existing grid search
for (seq, hid, lr, nl, bs, dr, ep) in sampled:
    print(f"→ Running random config: seq={seq}, hid={hid}, lr={lr}, layers={nl}, bs={bs}, dr={dr}, ep={ep}")
    run_pipeline_with_grid_search(
        df=df_wind,
        timestamp_col='TimeStamp',
        feature_cols=[
            "VoltL1_Mean","VoltL2_Mean","VoltL3_Mean",
            "CurrentL1_Mean","CurrentL2_Mean","CurrentL3_Mean",
            "PowerFactor_Mean","Frequency_Mean","WindSpeed_Mean",
            "hour_sin","hour_cos","minute_sin","minute_cos"
        ],
        target_col='ActivePower_Mean',
        sequence_length_options=[seq], 
        hidden_size_options=[hid],
        learning_rate_options=[lr],
        num_layers_options=[nl],
        batch_size_options=[bs],
        dropout_rate_options=[dr],
        epoch_options=[ep],
        log_path='rand_search_log_nowcast.csv',
        model_dir='rand_saved_models_nowcast'
    )

### Run & Train Best Model

In [14]:
run_pipeline_with_grid_search(
    df=df_wind,
    timestamp_col='TimeStamp',
    feature_cols=["VoltL1_Mean", "VoltL2_Mean", "VoltL3_Mean",
    "CurrentL1_Mean", "CurrentL2_Mean", "CurrentL3_Mean", "PowerFactor_Mean",
    "Frequency_Mean", "WindSpeed_Mean", "hour_sin", "hour_cos", "minute_sin", "minute_cos"],
    target_col='ActivePower_Mean',
    sequence_length_options=[96], 
    hidden_size_options=[128],
    learning_rate_options=[0.0003],
    num_layers_options=[3],
    batch_size_options=[32],
    dropout_rate_options=[0.2],
    epoch_options=[100],
    log_path='test_grid_log_final.csv',
    model_dir='test_saved_model_final'
    )


Generating sequences for sequence_length = 96
Trying config: seq96_hid128_lr0.0003_nl3_bs32_dr0.2_ep100_20250510_223729


W0000 00:00:1746909449.770399   15228 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 737s 83ms/step - loss: 0.0043 - val_loss: 7.0920e-05
Epoch 2/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 757s 85ms/step - loss: 1.0559e-04 - val_loss: 4.7878e-05
Epoch 3/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 740s 83ms/step - loss: 5.1545e-05 - val_loss: 3.0526e-05
Epoch 4/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 727s 82ms/step - loss: 3.6129e-05 - val_loss: 2.4446e-05
Epoch 5/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 725s 81ms/step - loss: 2.9707e-05 - val_loss: 1.6130e-05
Epoch 6/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 761s 85ms/step - loss: 2.5976e-05 - val_loss: 2.8094e-05
Epoch 7/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 755s 85ms/step - loss: 2.3262e-05 - val_loss: 2.0806e-05
Epoch 8/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 741s 83ms/step - loss: 2.3978e-05 - val_loss: 1.3511e-05
Epoch 9/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 750s 84ms/step - loss: 2.1925e-05 - val_loss: 2.1381e-05
Epoch 10/100
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 732s 82ms/step - loss: 2.0423e-05 - val_loss: 1.6605e-05
Epo

'test_grid_log_final.csv'